In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime 
import sys
%matplotlib inline
evaluation=pd.read_csv('submit.txt')
product_info=pd.read_csv('product_info.txt')
product_quantity=pd.read_csv('product_quantity.txt')

In [2]:
product_quantity['product_month']=product_quantity['product_date'].apply(lambda x: x[:7])
train_month=product_quantity.groupby(['product_id','product_month']).sum()['ciiquantity']

In [3]:
train_month.unstack()

product_month,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,...,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11
product_id,,,,,,,,,,,,,,,,,,,,,
1,29.0,111.0,13.0,71.0,74.0,30.0,55.0,159.0,35.0,134.0,...,39.0,102.0,283.0,136.0,52.0,85.0,48.0,37.0,102.0,85.0
2,46.0,59.0,104.0,144.0,167.0,117.0,194.0,148.0,138.0,109.0,...,182.0,424.0,165.0,192.0,262.0,94.0,112.0,196.0,222.0,169.0
3,29.0,41.0,88.0,72.0,159.0,143.0,57.0,125.0,497.0,104.0,...,201.0,59.0,125.0,289.0,288.0,116.0,147.0,250.0,237.0,310.0
4,72.0,115.0,117.0,121.0,115.0,103.0,92.0,92.0,92.0,187.0,...,95.0,83.0,66.0,217.0,156.0,94.0,26.0,17.0,26.0,106.0
5,98.0,40.0,106.0,72.0,105.0,132.0,88.0,120.0,65.0,146.0,...,138.0,244.0,98.0,135.0,104.0,228.0,98.0,170.0,120.0,102.0
6,41.0,86.0,207.0,256.0,263.0,157.0,143.0,181.0,151.0,188.0,...,150.0,374.0,200.0,186.0,79.0,287.0,270.0,63.0,454.0,143.0
7,45.0,11.0,3.0,22.0,96.0,37.0,116.0,98.0,81.0,190.0,...,114.0,242.0,257.0,121.0,24.0,223.0,116.0,35.0,160.0,57.0
8,61.0,111.0,101.0,69.0,112.0,74.0,143.0,134.0,75.0,96.0,...,91.0,131.0,156.0,160.0,110.0,141.0,162.0,185.0,257.0,208.0
9,43.0,62.0,111.0,127.0,275.0,62.0,86.0,53.0,96.0,84.0,...,45.0,110.0,81.0,106.0,29.0,134.0,90.0,17.0,90.0,101.0


In [4]:
train_month=pd.DataFrame(train_month)

In [5]:
train_month=train_month.reset_index()

In [6]:
train_month.head()

,product_id,product_month,ciiquantity
0,1,2014-01,29
1,1,2014-02,111
2,1,2014-03,13
3,1,2014-04,71
4,1,2014-05,74


# 产品特征预处理

In [7]:
def regularization(table,name):
    mean, std = table[name].mean(), table[name].std()
    table.loc[:, name] = (table[name] - mean)/std
    return table

In [8]:
def one_hot(table,name):
    dummies = pd.get_dummies(table[name], prefix=name, drop_first=False)
    table = pd.concat([table, dummies], axis=1)
    table = table.drop(name, axis=1)
    return table

In [9]:
'''
product_info_use=product_info.drop(['district_id1', 'district_id2', 'district_id3', 'district_id4', 'lat', 'lon', 'railway'
                                   , 'airport', 'citycenter', 'railway2', 'airport2','citycenter2', 'eval', 'eval2', 'eval3', 'eval4'
                                   , 'voters', 'startdate', 'upgradedate', 'cooperatedate', 'maxstock'],axis=1)
#'''
product_info_use=product_info.drop(['district_id3', 'district_id4', 'lat', 'lon', 'railway'
                                    , 'airport', 'citycenter', 'railway2', 'airport2','citycenter2'
                                    , 'eval', 'eval2', 'eval3', 'startdate', 'upgradedate', 'cooperatedate'],axis=1)
product_info_use.head()

,product_id,district_id1,district_id2,eval4,voters,maxstock
0,1,10201,20502,3.1,1034.0,75.0
1,2,10201,20502,3.4,1707.0,172.0
2,3,10201,20502,3.6,1739.0,188.0
3,4,10201,20502,3.5,1065.0,116.0
4,5,10201,20502,3.4,2209.0,95.0


In [10]:
for x in ['eval4', 'voters' ,'maxstock']:
    product_info_use=regularization(product_info_use,x)
product_info_use.head()

,product_id,district_id1,district_id2,eval4,voters,maxstock
0,1,10201,20502,-0.830884,-0.266098,-0.529165
1,2,10201,20502,-0.111140,0.063709,0.020579
2,3,10201,20502,0.368689,0.079391,0.111258
3,4,10201,20502,0.128774,-0.250906,-0.296799
4,5,10201,20502,-0.111140,0.309717,-0.415815


In [11]:
for x in ['district_id1', 'district_id2']:
    product_info_use=one_hot(product_info_use,x)
product_info_use.head()

,product_id,eval4,voters,maxstock,district_id1_10201,district_id1_10302,district_id1_10403,district_id1_10504,district_id1_11514,district_id1_11615,...,district_id2_1148418,district_id2_1150458,district_id2_1151274,district_id2_1151376,district_id2_1153722,district_id2_1157394,district_id2_1160352,district_id2_1187790,district_id2_1193808,district_id2_1307640
0,1,-0.830884,-0.266098,-0.529165,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,-0.111140,0.063709,0.020579,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0.368689,0.079391,0.111258,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0.128774,-0.250906,-0.296799,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,-0.111140,0.309717,-0.415815,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 训练数据预处理

In [12]:
def Pretreatment(table):
    table['year']=table['product_month'].apply(lambda x:(float(x[0:4])-2015.5)/4)
    table['month']=table['product_month'].apply(lambda x:x[5:7])
    table=one_hot(table,'month')
    table=table.drop('product_month', axis=1)    
    return table
Pretreatment(train_month).head()

,product_id,ciiquantity,year,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,1,29,-0.375,1,0,0,0,0,0,0,0,0,0,0,0
1,1,111,-0.375,0,1,0,0,0,0,0,0,0,0,0,0
2,1,13,-0.375,0,0,1,0,0,0,0,0,0,0,0,0
3,1,71,-0.375,0,0,0,1,0,0,0,0,0,0,0,0
4,1,74,-0.375,0,0,0,0,1,0,0,0,0,0,0,0


In [13]:
train_month=Pretreatment(train_month)
train_month.head()

,product_id,ciiquantity,year,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,1,29,-0.375,1,0,0,0,0,0,0,0,0,0,0,0
1,1,111,-0.375,0,1,0,0,0,0,0,0,0,0,0,0
2,1,13,-0.375,0,0,1,0,0,0,0,0,0,0,0,0
3,1,71,-0.375,0,0,0,1,0,0,0,0,0,0,0,0
4,1,74,-0.375,0,0,0,0,1,0,0,0,0,0,0,0


In [14]:
evaluation=Pretreatment(evaluation)
evaluation.head()

,product_id,ciiquantity_month,year,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,1,100,-0.125,0,0,0,0,0,0,0,0,0,0,0,1
1,2,100,-0.125,0,0,0,0,0,0,0,0,0,0,0,1
2,3,100,-0.125,0,0,0,0,0,0,0,0,0,0,0,1
3,4,100,-0.125,0,0,0,0,0,0,0,0,0,0,0,1
4,5,100,-0.125,0,0,0,0,0,0,0,0,0,0,0,1


In [16]:
train_month=pd.merge(train_month,product_info_use,on='product_id',how='left')
evaluation=pd.merge(evaluation,product_info_use,on='product_id',how='left')
train_month=train_month.drop('product_id',axis=1)
evaluation=evaluation.drop('product_id',axis=1)

In [17]:
print(train_month.shape,evaluation.shape)

(60557, 99) (56000, 99)


# 神经网络训练

In [23]:
def random_read_train_data(table,targets):    
    _features = table.drop(targets,axis=1).as_matrix()
    _targets = table[targets].as_matrix()
    _targets.shape = (_targets.shape[0], 1)
    _targets.transpose()
    return _features,_targets

In [24]:
train_month_x,train_month_y = random_read_train_data(train_month,'ciiquantity')
evaluation_x,evaluation_y = random_read_train_data(evaluation,'ciiquantity_month')
print(train_month_x.shape,train_month_y.shape)
print(evaluation_x.shape,evaluation_y.shape)

(60557, 98) (60557, 1)
(56000, 98) (56000, 1)


In [26]:
def add_layer(inputs, in_size, out_size, activation_function=None):    
    Weights = tf.Variable(tf.random_normal([in_size, out_size],mean=0.0,stddev=0.1))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs
in_shape=train_month_x.shape[1]
hidden_shape1=50
hidden_shape2=25
x = tf.placeholder(tf.float32, shape=[None, in_shape])
y = tf.placeholder(tf.float32, [None,1])
l1 = add_layer(x, in_shape, hidden_shape1, activation_function=tf.nn.sigmoid)
l2 = add_layer(l1, hidden_shape1, hidden_shape2, activation_function=tf.nn.sigmoid)
prediction = add_layer(l2, hidden_shape2, 1, activation_function=None)
loss = tf.sqrt(tf.reduce_mean(tf.reduce_sum(tf.square(y - prediction),
                     reduction_indices=[1])))
saver = tf.train.Saver()
sess=tf.Session()
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [27]:
loss_value=sess.run(loss, feed_dict={x: train_month_x, y: train_month_y})
print('Loss: {:.6f}'.format(loss_value))
epochs=3000
l_r=0.01
start = datetime.now()
train_step = tf.train.GradientDescentOptimizer(l_r).minimize(loss)
for epoch in range(epochs):    
    sess.run(train_step, feed_dict={x: train_month_x, y: train_month_y})
    if epoch % 100 == 0:
        temploss = sess.run(loss, feed_dict={x: train_month_x, y: train_month_y})
        end = datetime.now()
        sys.stdout.write('\rEpoch {:>2} Loss: {:.6f} time: {}'.format(epoch + 1,temploss,end-start))
        start = datetime.now()

Loss: 299.699463
Epoch 2901 Loss: 256.496643 time: 0:00:08.866190

In [32]:
loss_value=sess.run(loss, feed_dict={x: train_month_x, y: train_month_y})
answer_table=pd.read_csv('submit.txt')
answer_table.ciiquantity_month=sess.run(prediction, feed_dict={x: evaluation_x})
answer_table.to_csv('nn_{:.3f}.csv'.format(loss_value),index=False)
print(loss_value)

255.971
